<a href="https://colab.research.google.com/github/ekqlsrla/ESAA-2/blob/main/SESSION/0926_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표기는 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것이고, 간접 투표는 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측하는 것이다. 이 방식은 확률이 높은 투표에 비중을 더 두기 떄문에 직접 투표 방식보다 성능이 높다. (P249)

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

훈련 데이터에 과대 적합되었다면 **학습률을 감소**시켜야 한다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [29]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [30]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val,X_test,y_train_val,y_test = train_test_split(X,y,random_state = 42)


In [31]:
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val,random_state = 42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [32]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [33]:
# model fitting

rfc = RandomForestClassifier(n_estimators = 100, random_state = 42)
rfc.fit(X_train,y_train)

extra = ExtraTreesClassifier(n_estimators = 100, random_state = 42)
extra.fit(X_train,y_train)

lin_svc = LinearSVC(max_iter = 100, random_state = 42)
lin_svc.fit(X_train,y_train)

mlp = MLPClassifier(random_state = 42)
mlp.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [34]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators = [('rf',rfc),('extra',extra),('svc',lin_svc),('MLP',mlp)],
    voting = 'hard'
)

In [35]:
# model fitting

voting_clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('svc', LinearSVC(max_iter=100, random_state=42)),
                             ('MLP', MLPClassifier(random_state=42))])

In [36]:
# model test
voting_clf.score(X_val,y_val)


0.9696761904761905

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `set_params`를 활용해보세요

In [8]:
# 각 분류 모델 학습



In [37]:
# 각 분류 모델의 성능 확인
for clf in (rfc,lin_svc,mlp,extra) :
  print(clf.score(X_val,y_val))


0.9651047619047619
0.7859809523809523
0.9639619047619048
0.9697523809523809


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. svm분류기의 성능이 가장 낮다

In [40]:
# 가장 성능이 낮은 모델 제거

voting_clf.set_params(svc = 'drop')

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('svc', 'drop'),
                             ('MLP', MLPClassifier(random_state=42))])

In [ ]:
del voting_clf.estimators_[2]

In [41]:
# model fitting

voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('svc', 'drop'),
                             ('MLP', MLPClassifier(random_state=42))])

In [42]:
# 모델 제거 후 성능 확인
voting_clf.score(X_val,y_val)


0.9716571428571429

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [44]:
import numpy as np
estimators = [rfc,lin_svc,mlp,extra]
X_val_pred = np.empty((len(X_val), 4), dtype=np.float32)
for i, estimators in enumerate(estimators) :
  X_val_pred[:,i] = estimators.predict(X_val)



####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [45]:
rfc_clf = RandomForestClassifier(random_state = 42)
rfc_clf.fit(X_val_pred, y_val)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [57]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_pred = np.empty((len(X_test), 4), dtype=np.float32)
estimators = [rfc,lin_svc,mlp,extra]
for i,estimator in enumerate(estimators) :
  X_test_pred[:,i] = estimator.predict(X_test)

In [58]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rfc_clf.predict(X_test_pred)

In [61]:
# model test
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.9666285714285714